In [1]:
import torch
import seaborn as sns
import pandas as pd
import transformers
import evaluate
from transformers import pipeline, AutoTokenizer
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset, load_from_disk
from collections import Counter, defaultdict
from transformers import DataCollatorWithPadding
from pathlib import Path
import numpy as np
from sklearn.metrics import f1_score, classification_report, accuracy_score
from tqdm.auto import tqdm
import re
from nltk.tokenize import sent_tokenize
from sklearn.linear_model import LinearRegression
sns.set()

In [2]:
#dataset = load_from_disk('/datadrive_2/')
#test_data = dataset['test']

cache_dir = '/datadrive_2/hf_cache/'
dataset = load_from_disk("/datadrive_2/HMD_chunked_100_test/")
dataset #= dataset['train']

Dataset({
    features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr'],
    num_rows: 11315511
})

## Classify by Political Leaning

In [3]:
pol_pattern = re.compile(r'\bliberal|\bconservat|\btory\b|\btories\b',re.I)

In [4]:
pol_pattern.findall('liberal governments do not fire their conservative political ministers minister')

['liberal', 'conservat']

In [5]:
# def sent_split(x):
#      return {'data': [
#                 {'sentence':s.lower(),
#                  'length': len(s.split()),
#                  'pol': p, 'loc':l, 'year':y, 'ocr':o,'nlp':n} 
#                      for y,p,l,o,n,t in zip(x['year'],x['pol'],x['loc'],x['ocr_quality_mean'],x['nlp'],x['text']) 
#                       for s in sent_tokenize(t) 
#                          if pol_pattern.findall(s)
#                  ]
#             }

# test_data = dataset.map(sent_split,batched=True, remove_columns=dataset.column_names)

In [6]:
test_data = dataset.map(
                lambda x: {'sentences': x['sentences'].lower()}, num_proc=6
                        ).shuffle(seed=42).select(range(15000))

Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-6f3d8b9a1021878e.arrow
Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-2d79338e1133aae6.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-24d2a5525e368df0.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-b3061892df3a5bbe.arrow
Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-06bf12bcec6754fc.arrow
Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-a090791e087fa494.arrow
Loading cached shuffled indices for dataset at /datadrive_2/HMD_chunked_100_test/cache-2e3366e450e9f2f2.arrow


In [7]:
test_data

Dataset({
    features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr'],
    num_rows: 15000
})

In [8]:
#data = test_data.filter(lambda x: x['data.length'] > 25).shuffle(seed=42).select(range(15000))

In [9]:
# def pred_data(example,add_field='year'):
#     return {'st_year_sep': f'[{example[add_field]}]' + ' [SEP] ' + example['sentences'] ,
#      'year_sep': str(example[add_field]) + ' [SEP] ' + example['sentences'] ,
#      'year_date': str(example[add_field]) + ' [DATE] ' + example['sentences'],
        
#     }
def pred_data(example):
    return {'label':float(example['year'])}
    
data = test_data.map(pred_data , num_proc=6)

Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-ff0fd48ef7c4aa88.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-6aeec3fc41d9a975.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-0f45e64b62b46afe.arrow
Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-a1b252da72d25d12.arrow


Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-b1a1fc69d546bff6.arrow
Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-4bbfb9a6d353944e.arrow


In [10]:
len(data)

15000

In [11]:
# lab2code = {'[con]':0,'[lib]':1,'[rad]':2,'[neutr]':3,'[none]':4}
# num_labels = len(lab2code)
# data = data.map(lambda x: {'label': lab2code[x['pol']]})

In [12]:
data[0]

{'year': 1859,
 'nlp': 2084,
 'pol': '[neutr]',
 'loc': '[liverpool]',
 'sentences': 'sitenirranv, roy institute as tou ry, v. ireno. i.—nervous debility, loss of kient.„2,l:oo!op orof sight, prostration of physical energy, 1;\': asor\'\'business, study, or society, with dr. marston \'01coax. men.o• heiti 0addressed specially to young 14. ...e oh 0%—marrlagerbanodbilgaltetipoltnsodanudetilrn°ooimp ten &c. \' addressed to those who dam. 1010 . totof removing all impediments. qi thee dirpthaenß:orrrs4 2ar.cll\'of healthful children, clearly expl.-11:91p .no. 3.—tiie great social evil. 11\' wee voerdiseases which result from it, with da. minor ~,fc „i;,da. massrosi continues to be consulted pe ,e,:.„penned with.veoftiit.reatment, by which msactinr luso* a\'\'\'s,y.o,,tifi,,:vilti9.residence,ez ens-......r5z 0 d-6 orby letter,47, eitherin at thexrßotal.lnsroittitatetßa: 1171,01\'"raftpaordrticeuiveargry.inf°""ti°l4 70-00.',
 'ocr': 0.643,
 'label': 1859.0}

In [13]:
test_size = int(len(data)*.2)
train_test = data.train_test_split(test_size=test_size, seed=1984)
test_set = train_test['test']
val_size = int(len(train_test['train'])*.15)
train_val =  train_test['train'].train_test_split(test_size=val_size, seed=1984)

Loading cached split indices for dataset at /datadrive_2/HMD_chunked_100_test/cache-772543886c89d7f3.arrow and /datadrive_2/HMD_chunked_100_test/cache-e018be346372ac2c.arrow
Loading cached split indices for dataset at /datadrive_2/HMD_chunked_100_test/cache-0f7c9122ef09d4a9.arrow and /datadrive_2/HMD_chunked_100_test/cache-4947f263d6c6457a.arrow


In [14]:
train_val

DatasetDict({
    train: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr', 'label'],
        num_rows: 10200
    })
    test: Dataset({
        features: ['year', 'nlp', 'pol', 'loc', 'sentences', 'ocr', 'label'],
        num_rows: 1800
    })
})

In [15]:
checkpoints = [('distilbert','distilbert-base-uncased','[SEP]','sentences'),
               ('hmd_distilbert','/datadrive_2/bnert-hmd','[SEP]','sentences'),
               #('bnert-time-st-y','/datadrive_2/bnert-time-st-y','[SEP]','sentences'),
               ('bnert-time-y','/datadrive_2/bnert-time-y','[DATE]','sentences'),
               ('bnert-time-y_masked_25','/datadrive_2/bnert-time-y_masked_25','[DATE]','sentences'),
               ('bnert-time-y_masked_75','/datadrive_2/bnert-time-y_masked_75','[DATE]','sentences'),
               #('bnert-pol-st','/datadrive_2/bnert-pol-st','[SEP]','year_sep'),
               #('bnert-pol','/datadrive_2/bnert-pol','[SEP]','sentences')
              ]

model_dict = defaultdict(dict)
for name,checkpoint, st, sent_col in checkpoints:
    model_dict[name]['model'] = AutoModelForSequenceClassification.from_pretrained(checkpoint,num_labels=1)
    model_dict[name]['tokenizer'] = AutoTokenizer.from_pretrained(checkpoint)
    #model_dict[name]['special_token'] = st
    model_dict[name]['sentences'] = sent_col

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'classifier

In [16]:
train_val = train_val.remove_columns(['nlp', 'ocr', 'loc'])
train_val

DatasetDict({
    train: Dataset({
        features: ['year', 'pol', 'sentences', 'label'],
        num_rows: 10200
    })
    test: Dataset({
        features: ['year', 'pol', 'sentences', 'label'],
        num_rows: 1800
    })
})

In [17]:
#def add_text_col(example,source):
#    return {'text' : example[source]}

def preprocess_function(examples, target_col):
    return tokenizer(examples[target_col], truncation=True, padding="max_length", max_length=256)

In [18]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def compute_metrics_for_regression(eval_pred):
    logits, labels = eval_pred
    labels = labels.reshape(-1, 1)
    #print(labels, logits)
    mse = mean_squared_error(labels, logits)
    mae = mean_absolute_error(labels, logits)
    r2 = r2_score(labels, logits)
    single_squared_errors = ((logits - labels).flatten()**2).tolist()
    
    # Compute accuracy 
    # Based on the fact that the rounded score = true score only if |single_squared_errors| < 0.5
    accuracy = sum([1 for e in single_squared_errors if e < 0.25]) / len(single_squared_errors)
    
    return {"mse": mse, "mae": mae, "r2": r2, "accuracy": accuracy}

class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs[0][:, 0]
        loss = torch.nn.functional.mse_loss(logits, labels)
        return (loss, outputs) if return_outputs else loss

In [19]:
result_dict = defaultdict(dict)

for name, mdict in model_dict.items():
    print(f'Creating a model for {name}')
    tokenizer = model_dict[name]['tokenizer']
    model = model_dict[name]['model']
    data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
    sent_col = model_dict[name]['sentences']
    
    #train_val = train_val.map(add_text_col,fn_kwargs={'source': sent_col})
    train_val = train_val.map(preprocess_function,fn_kwargs={'target_col': sent_col})
    
    training_args = TrainingArguments(
    seed=1984,
    evaluation_strategy="epoch",
    output_dir=f"../results_{name}",
    learning_rate=1e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=40,
    weight_decay=0.05,
        )

#     trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=train_val["train"],
#     eval_dataset=train_val["test"],
#     tokenizer=tokenizer,
#     data_collator=data_collator,
#         )


#     trainer.train()

    trainer = RegressionTrainer(
        model=model,
        args=training_args,
        train_dataset=train_val["train"],
        eval_dataset=train_val["test"],
        compute_metrics=compute_metrics_for_regression,
        )

    trainer.train()
    test_set = test_set.map(preprocess_function,fn_kwargs={'target_col': sent_col})
    trainer.eval_dataset=test_set
    trainer.evaluate()
    scores = trainer.evaluate()
    #print(scores)
    model.save_pretrained(f'/datadrive_2/{name}-time-reg')
    tokenizer.save_pretrained(f"/datadrive_2/{name}-time-reg")
    
    
    result_dict[name]['loss'] = scores['eval_loss']
    result_dict[name]['mae'] = scores['eval_mae']
    result_dict[name]['mse'] = scores['eval_mse']
    result_dict[name]['accuracy']  = scores['eval_accuracy']

Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-ad2d056fe0e39963.arrow
Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-d25d1d2e1389c0e8.arrow


Creating a model for distilbert


The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentences, pol, year. If sentences, pol, year are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
/datadrive_2/lm2/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 10200
  Num Epochs = 40
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 51000


Epoch,Training Loss,Validation Loss,Mse,Mae,R2,Accuracy
1,3302758.400000,3214710.500000,3214711.000000,1792.879395,-10940.692318,0.000000
2,2978440.448000,2888648.750000,2888648.750000,1699.516113,-9830.897933,0.000000
3,2642760.704000,2413907.750000,2413907.750000,1553.580811,-8215.052705,0.000000
4,1990798.336000,1821300.875000,1821300.875000,1349.446899,-6198.038032,0.000000
5,1494567.168000,1177249.125000,1177249.125000,1084.875732,-4005.922946,0.000000
6,755149.248000,578930.375000,578930.375000,760.681641,-1969.466011,0.000000
7,349306.176000,143939.171875,143939.187500,379.005768,-488.915990,0.000000
8,15369.130000,307.340149,307.340179,13.505940,-0.046073,0.023333
9,2639.978800,336.012054,336.012085,13.728494,-0.143661,0.029444
10,2731.486000,295.118347,295.118378,13.538379,-0.004474,0.021111


Saving model checkpoint to ../results_distilbert/checkpoint-1000
Configuration saved in ../results_distilbert/checkpoint-1000/config.json
Model weights saved in ../results_distilbert/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentences, pol, year. If sentences, pol, year are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1800
  Batch size = 8
Saving model checkpoint to ../results_distilbert/checkpoint-1500
Configuration saved in ../results_distilbert/checkpoint-1500/config.json
Model weights saved in ../results_distilbert/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ../results_distilbert/checkpoint-2000
Configuration saved in ../results_distilbert/checkpoint-2000/config.json
Model weights saved in ../results_distilbert/checkpoint-

...

***** Running Evaluation *****
  Num examples = 3000
  Batch size = 8


The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: sentences, year, ocr, loc, pol, nlp. If sentences, year, ocr, loc, pol, nlp are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3000
  Batch size = 8
Configuration saved in /datadrive_2/hmd_distilbert-time-reg/config.json
Model weights saved in /datadrive_2/hmd_distilbert-time-reg/pytorch_model.bin
tokenizer config file saved in /datadrive_2/hmd_distilbert-time-reg/tokenizer_config.json
Special tokens file saved in /datadrive_2/hmd_distilbert-time-reg/special_tokens_map.json
Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-cead509d66e37495.arrow
Loading cached processed dataset at /datadrive_2/HMD_chunked_100_test/cache-a23699cca14a5879.arrow
PyTorch: setting up devices
The default value for the training argumen

Creating a model for bnert-time-y


Epoch,Training Loss,Validation Loss,Mse,Mae,R2,Accuracy
1,3296848.640000,3206718.250000,3206718.500000,1790.649170,-10913.489351,0.000000
2,2969220.096000,2879951.250000,2879951.750000,1696.955444,-9801.295608,0.000000
3,2636109.824000,2410068.750000,2410068.500000,1552.344849,-8201.986624,0.000000
4,1993128.960000,1825652.375000,1825652.375000,1351.058228,-6212.848543,0.000000
5,1502860.032000,1190133.000000,1190133.000000,1090.797485,-4049.774819,0.000000
6,772839.360000,596766.562500,596766.625000,772.316528,-2030.174087,0.000000
7,366656.416000,158582.968750,158582.968750,397.855682,-538.758122,0.000000
8,20544.850000,299.933472,299.933472,14.517337,-0.020863,0.020000
9,2700.289200,297.162842,297.162872,13.571680,-0.011433,0.035000
10,2822.433300,317.207855,317.207855,13.552204,-0.079659,0.026111


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

Saving model checkpoint to ../results_bnert-time-y/checkpoint-1500
Configuration saved in ../results_bnert-time-y/checkpoint-1500/config.json
Model weights saved in ../results_bnert-time-y/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to ../results_bnert-time-y/checkpoint-2000
Configuration saved in ../results_bnert-time-y/checkpoint-2000/config.json
Model weights saved in ../results_bnert-time-y/checkpoint-2000/pytorch_model.bin
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current value

In [20]:
results_df = pd.DataFrame.from_dict(result_dict, orient='index')

In [21]:
print(results_df.round(3).to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &     loss &     mae &      mse &  accuracy \\
\midrule
distilbert     &  202.908 &  11.098 &  202.908 &     0.028 \\
hmd\_distilbert &  210.152 &  11.300 &  210.152 &     0.031 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_1100391/2842510611.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results_df.round(3).to_latex())


In [ ]:
results_df.to_csv('tables/classsify_time_2.csv')

In [23]:
import pandas as pd
results_df = pd.read_csv('tables/classsify_time_1.csv')
results_df

,Unnamed: 0,loss,mae,mse,accuracy
0,distilbert,354.605835,15.147977,354.605835,0.021667
1,hmd_distilbert,354.234924,15.323492,354.234924,0.021667
2,bnert-time-y,313.964966,14.556096,313.964966,0.018000
3,bnert-time-y_masked_25,323.114471,14.468663,323.114471,0.020333
4,bnert-time-y_masked_75,338.570129,15.287087,338.570129,0.014333


In [ ]:
!ls -la tables/